In [1]:
import json
import string
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import re
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM, AutoModel, AutoModelForQuestionAnswering,  T5Tokenizer, T5Model
import torch
import torch.nn.functional as F
!pip install -U transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data = pd.read_json('/content/drive/MyDrive/NLP/Manually_Extracted_Data.jsonl', lines=True)

In [3]:
print("targetParagraphs:")
print(data['targetParagraphs'])

targetParagraphs:
0      DEARBORN, Mich. - Ford, the only U.S. automake...
1      The President of the Swiss Confederation, Mrs....
2      The discovery of the presence of airborne asbe...
3      On October 14, 2008, Canadians will be heading...
4      ATHENS, Jan. 12 — An antitank grenade was fire...
                             ...                        
101    11-year-old Chris Gaither was home alone when ...
102    It seems Mother Monster takes her moniker to h...
103    What the GOOP just came out of your mouth, Gwy...
104    Get ready to finally embrace your short statur...
105    ANNISTON ARMY DEPOT, Alabama — Used in nearly ...
Name: targetParagraphs, Length: 106, dtype: object


In [4]:
sentences = data['targetParagraphs'].iloc[10]
postTexts = data['postText'].iloc[10]
#paragraph = " ".join(sentences)
paragraph = str(sentences)
paragraph = paragraph.replace('"', '')
postText = str(postTexts)
print(paragraph)
print(postText)


A woman passenger who committed suicide in a train toilet by setting herself on fire was named today as Rachel James.The blonde passenger, 43, from Uxbridge, west London, calmly kissed an elderly woman before setting herself on fire, it was revealed today.Carrying a can of highly flammable gas, she walked along an aisle on the London-bound train as it approached Leighton Buzzard railway station, in Bedfordshire, before igniting it in the cubicle.Blaze: The 43-year-old passenger was seen kissing an elderly woman moments before fire broke out in a cubicle on the London-bound train.The 43-year-old was seen kissing an elderly woman moments before a fire broke out in a cubicle on the London-bound train.Passengers were forced to flee the resulting blaze after hearing a loud explosion on the 4.25pm service from Northampton on Monday.Horrific scenes outside the Trump trial as a man sets himself on FIRE.Tim Naylor, who was evacuated from the train, said: 'There was a big, loud screaming sound.'

In [5]:
def generate_title(paragraph):
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
  model_name = "JulesBelveze/t5-small-headline-generator"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = T5ForConditionalGeneration.from_pretrained(model_name)

  input_ids = tokenizer(
      [WHITESPACE_HANDLER(paragraph)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=384
  )["input_ids"]

  output_ids = model.generate(
      input_ids=input_ids,
      max_length=84,
      no_repeat_ngram_size=2,
      num_beams=4
  )[0]

  summary = tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )

  return summary

In [6]:
def generate_titles():
  for para in data['targetParagraphs']:
    paragraph = str(para)
    title = generate_title(paragraph)
    return title
    print(title)

In [7]:
generate_titles()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'Ford’s first annual profit in four years was a $118.3 billion, CEO Alan Mulally says'

Take attention mask into account for correct averaging

In [8]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")

def generate_questions(text):
    inputs = tokenizer.encode("question: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, early_stopping=True)
    questions = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return questions



tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [9]:
# Load tokenizer and model for question answering
qa_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def answer_question(question, context):
    inputs = qa_tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = qa_model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = qa_tokenizer.decode(input_ids[answer_start:answer_end], skip_special_tokens=True, decoder_inputs_ids=inputs)
    return answer

In [13]:
#for generating title from paragraph and generating questions

def generateQueT():
  titlesQue = []
  for para in(data['targetParagraphs']):
    para = str(para)
    #generate title
    title = generate_title(para);

    # Generate questions for genereated titiles
    questionsT = generate_questions(title)
    titlesQue.append(questionsT)
    #print(questionsT)
  print("finish")
  return titlesQue

In [14]:
generateQueT()

finish


['What was Ford’s first profit in four years?',
 "What is the name of the Swiss Confederation President's visit to Ghana?",
 'What is the first concern of the Montclair Public Schools?',
 'What did Wikinews contact Liz White to talk about the riding?',
 'What happened to the US Embassy?',
 'What is the plan to fight?',
 'London mayor says he wants to end the ban on gay marriage',
 'The New York Times By BARRY MEIER',
 'What was the most common time a teenager watched television on a Saturday?',
 "What is the name of the contest you've been competing in?",
 'Rachel James set herself on fire in a train toilet?',
 'What has intensified the political crisis in Madagascar?',
 'What has become like foreign movies?',
 "What is the Mets fan's whining about?",
 "What is the cause of the detainee's hunger strike?",
 'What is the name of the event Gay is trying to win?',
 'Wall Street soaring for the second time this week?',
 'The new entry will be peer reviewed separately before it is published 

In [15]:
#for generating question for posttext

def generateQueP():
  #generating quetion postText
  postTextQu = []
  for post in(data['postText']):
    postText = str(post)

    # Generate questions for genereated titiles
    questionsP = generate_questions(postText)
    postTextQu.append(questionsP)
    #print(questionsT)
  return postTextQu

In [16]:
generateQueP()

['Ford Motors posts a profit of about US$2.7 billion each year',
 'The Swiss President visits Ghana',
 'What triggers evacuation of a New Jersey middle school?',
 'What is the name of the candidate that is running in Toronto Centre?',
 'Explosion at the U.S. embassy in Athens',
 'The regional council fights to keep sustainable growth on the Sunshine Coast',
 'What is the name of the gay pride parade?',
 'What is the name of the missing ex-F.B.I. agent?',
 'What is the correlation between church attendance and adolescent obesity?',
 "What is Elena Congost Mohedano's sport?",
 'A passenger dies after a toilet fire on a train in Bedfordshire, England?',
 'What is the cause of the Mutiny?',
 'Eight people died in the bank heist in Iraq?',
 'Mets Still Star in the Theater of Absurdity',
 'Guantanamo prisoners are on a hunger strike?',
 'What is the name of the St. Jude Classic?',
 'Investors See a Glimmer and Send Shares Higher?',
 'RNA journal submits articles to Wikipedia',
 'Ayman al-Zaw

In [17]:
pos = data['postText'].iloc[2]
pos = str(pos)
posQT = generate_questions(pos)
post = str(posQT)
paraT = data['targetParagraphs'].iloc[2]
para = str(paraT)
para = str(para)
print(para)
print(post)

The discovery of the presence of airborne asbestos in a middle school in Montclair, New Jersey Friday prompted the evacuation of over 200 students from the school and the school's closure. Renaissance Middle School, part of the Montclair Public Schools in New Jersey, was closed Monday and remains closed Tuesday while undergoing asbestos testing and cleaning.According to a letter sent home to parents by the Superintendent's Office of Montclair Public Schools on Friday, "plaster may have been disturbed" during construction on new fire doors at the Renaissance School building on Thursday.An inspection arranged by the school district indicated asbestos was present in the plaster, and a subsequent inspection performed by asbestos consultant Detail Associates revealed "a level of airborne asbestos fibers that exceeded the acceptable range" in the third floor hallway of the school. Montclair Public Schools business administrator Dana Sullivan told The Star-Ledger that testing conducted on Mar

In [ ]:
ans = answer_question(post, para)
print("The answer is:", ans)

The answer is: airborne asbestos


In [18]:
#converting generated title and postText into questions
titleQu = generateQueT()
postQu = generateQueP()

finish


In [21]:
#finding answer for postText questions
def posAnswer():
  pQanswers = []
  for posQ, paraS in zip(postQu, data['targetParagraphs']):
    #finding answe
    posQT = posQ
    paraS = str(paraS)
    paraT = str(paraS)
    print(posQT)
    print(paraT)
    ans = answer_question(posQT, paraT)
    print("The answer is:", ans)
    pQanswers.append(ans)
  return pQanswers


In [22]:
#generating answer for all postText question
pQanswers = posAnswer()

Ford Motors posts a profit of about US$2.7 billion each year
DEARBORN, Mich. - Ford, the only U.S. automaker to avoid bankruptcy court, clawed its way to a US$2.7-billion profit in 2009 and expects to stay in the black in 2010. It was the automaker's first annual profit in four years.Ford's full-year revenue of $118.3 billion fell nearly 20 per cent from 2008, but the Dearborn-based automaker benefited from cost-cutting, a $696 million profit in its credit arm and popular cars and trucks like the Ford Fusion midsize sedan and Ford Escape small SUV. It gained market share in North and South America and Europe, despite the worst U.S. sales climate in 30 years."While we still face significant business environment challenges ahead, 2009 was a pivotal year for Ford," Ford CEO Alan Mulally said in a statement.Ford shares rose 32 cents, or nearly three per cent, to $11.87 in premarket trading.Ford's 2009 net income of 86 cents per share showed a significant improvement from the year before, w

In [23]:
#concattenating titles question and answer
def quesTitleAnswer():
  tQAnswer = []
  for tQ, ansr in zip(titleQu, pQanswers):
    tQT = str(tQ)
    concatenated_string = str(tQT + " " + ansr)
    print("the concat:", concatenated_string)
    tQAnswer.append(concatenated_string)
  return tQAnswer

In [24]:
#concatenated output
tQAnswe = quesTitleAnswer()

the concat: What was Ford’s first profit in four years? 2009
the concat: What is the name of the Swiss Confederation President's visit to Ghana? the president of the swiss confederation, mrs. micheline calmy - rey, has begun a 2 - day visit to ghana
the concat: What is the first concern of the Montclair Public Schools? airborne asbestos
the concat: What did Wikinews contact Liz White to talk about the riding? liz white
the concat: What happened to the US Embassy? an antitank grenade was fired into the heavily fortified american embassy here just before dawn today
the concat: What is the plan to fight? the regional council fights to keep sustainable growth on the sunshine coast
the concat: London mayor says he wants to end the ban on gay marriage what is the name of the gay pride parade?
the concat: The New York Times By BARRY MEIER robert levinson
the concat: What was the most common time a teenager watched television on a Saturday? a correlation between adolescent obesity amongst paci

score generation part

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [26]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [27]:
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min = 1e-9)

In [28]:
def get_sentence_embedding(sentence):
    encoded_input = tokenizer(sentence, padding = True, truncation = True, return_tensors = 'pt')

    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p = 2, dim = 1)

    return sentence_embeddings

In [68]:
def score():
  predicted_labels = []
  for pst, tqa in zip(postQu, tQAnswe):
    pstT = str(pst)
    tqaT = str(tqa)
    para_embedding = get_sentence_embedding(pstT)
    post_embedding = get_sentence_embedding(tqaT)
    similarity_score = F.cosine_similarity(para_embedding,  post_embedding).numpy()
    print("The postText question is:-", pstT)
    print("Generated Title and answer is:-",tqaT)
    print(similarity_score[0])
    if(similarity_score[0]<0.543):
      predicted_label = 1
      print("clickBaits")
    else:
      predicted_label = 0
      print("Not a clickBaits")
    predicted_labels.append(predicted_label)
  return predicted_labels

In [ ]:
s1 = "my father is a doctor"
s2 = "my father is a doctor"
para_embedding = get_sentence_embedding(s1)
post_embedding = get_sentence_embedding(s2)
similarity_score = F.cosine_similarity(para_embedding,  post_embedding).numpy()
#print("Generated Title and answer is:-",tqa)
#print("The postText question is:-", pst)
print(similarity_score[0])

In [ ]:
score()

In [32]:
import pandas as pd

data = pd.read_json('/content/drive/MyDrive/NLP/Manually_Extracted_Data.jsonl', lines=True)
true_labels = data['Clickbait'].tolist()

In [34]:
def calculate_accuracy(predicted_labels, true_labels):
  predicted_labels = np.array(predicted_labels)
  true_labels = np.array(true_labels)

  # Calculate accuracy by comparing predicted and true labels
  accuracy = np.mean(predicted_labels == true_labels)

  return accuracy

In [39]:
def calculate_precision(predicted_labels, true_labels):
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Calculate true positives (TP), false positives (FP), and precision
    true_positives = np.sum((predicted_labels == 1) & (true_labels == 1))
    false_positives = np.sum((predicted_labels == 1) & (true_labels == 0))

    if true_positives + false_positives == 0:
        precision = 0  # Handle division by zero
    else:
        precision = true_positives / (true_positives + false_positives)

    return precision


In [41]:
def calculate_recall(predicted_labels, true_labels):
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Calculate true positives (TP), false negatives (FN), and recall
    true_positives = np.sum((predicted_labels == 1) & (true_labels == 1))
    false_negatives = np.sum((predicted_labels == 0) & (true_labels == 1))

    if true_positives + false_negatives == 0:
        recall = 0  # Handle division by zero
    else:
        recall = true_positives / (true_positives + false_negatives)

    return recall

In [69]:
predicted_labels=[]
predicted_labels=score()
accuracy=calculate_accuracy(predicted_labels,true_labels)
precision=calculate_precision(predicted_labels,true_labels)
recall = calculate_recall(predicted_labels, true_labels)

The postText question is:- Ford Motors posts a profit of about US$2.7 billion each year
Generated Title and answer is:- What was Ford’s first profit in four years? 2009
0.73265386
Not a clickBaits
The postText question is:- The Swiss President visits Ghana
Generated Title and answer is:- What is the name of the Swiss Confederation President's visit to Ghana? the president of the swiss confederation, mrs. micheline calmy - rey, has begun a 2 - day visit to ghana
0.80187535
Not a clickBaits
The postText question is:- What triggers evacuation of a New Jersey middle school?
Generated Title and answer is:- What is the first concern of the Montclair Public Schools? airborne asbestos
0.49421406
clickBaits
The postText question is:- What is the name of the candidate that is running in Toronto Centre?
Generated Title and answer is:- What did Wikinews contact Liz White to talk about the riding? liz white
0.21161851
clickBaits
The postText question is:- Explosion at the U.S. embassy in Athens
Gen

In [70]:
print("Accuracy:",accuracy)
print("Precision:",precision)
print("Recall:", recall)

Accuracy: 0.6509433962264151
Precision: 0.6507936507936508
Recall: 0.7321428571428571
